In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import warnings
import import_ipynb
from scipy.stats import norm

warnings.filterwarnings('ignore')

from one_iv_computation import get_spot_and_risk_free_rate 
IV_surface_calls = pd.read_pickle('IV_surface_calls.pkl')
IV_surface_puts = pd.read_pickle('IV_surface_puts.pkl')


In [ ]:
def black_scholes(S, K, T, r, sigma, option_type):
    if T <= 0:
        if option_type == 'call':
            return max(S - K, 0)
        else:
            return max(K-S, 0)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option_type=='put':
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    else:
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def iv_to_price(iv_surface, S, r, option_type='call'):
    prices = iv_surface.copy()
    
    for exp in iv_surface.columns:
        T = exp   
        for K in iv_surface.index:
            sigma = iv_surface.loc[K, exp]
            if pd.notna(sigma) and sigma > 0:
                if option_type == 'call':
                    prices.loc[K, exp] = black_scholes(S, K, T, r, sigma,'call')
                else:
                    prices.loc[K, exp] = black_scholes(S, K, T, r, sigma,'put')
            else:
                prices.loc[K, exp] = np.nan
    
    return prices


In [ ]:
def detect_calendar_arbitrage(price_surface):
    """
    longer-dated options must be worth more
    """
    violations = []
    expirations = sorted(price_surface.columns)
    
    for strike in price_surface.index:
        for i in range(len(expirations) - 1):
            t1, t2 = expirations[i], expirations[i+1]
            p1 = price_surface.loc[strike, t1]
            p2 = price_surface.loc[strike, t2]
            
            if pd.notna(p1) and pd.notna(p2):
                if p1 > p2:
                    violations.append(f"Strike {strike} with time difference {t2-t1} and price difference {p1-p2}")
    
    return violations 

In [6]:
def detect_vertical_arbitrage(price_surface):
    """
    lower call strikes should have a higher price. 
    the price of a spread should never exceed the difference between strikes.
    """
    violations = []
    
    for expiry in price_surface.columns:
        prices = price_surface[expiry].dropna().sort_index()
        strikes = prices.index
        
        for i in range(len(strikes) - 1):
            k1, k2 = strikes[i], strikes[i+1]
            p1, p2 = prices[k1], prices[k2]
            
            if p2 > p1:
                violations.append(f"Strike difference {k2-k1} with price difference {p2-p1} (monotonicity)")
            
            spread_value = p1 - p2
            max_spread = k2 - k1
            if spread_value > max_spread:
                violations.append(f"Strike difference {k2-k1} with spread {spread_value} (above max {max_spread})")
    
    return violations 

In [13]:
def detect_butterfly_arbitrage(price_surface):
    """
    price of a butterfly should be positive. 
    """
    violations = []
    
    for exp in price_surface.columns:
        prices = price_surface[exp].dropna().sort_index()
        strikes = prices.index
        
        for i in range(len(strikes) - 2):
            k1, k2, k3 = strikes[i], strikes[i+1], strikes[i+2]
            p1, p2, p3 = prices[k1], prices[k2], prices[k3]
            
            butterfly = p1 - 2*p2 + p3

            if butterfly <= -0.05:
                violations.append(f"Butterfly cost is negative ({butterfly}) for expiration {exp} and strikes {k1}, {k2}, {k3}")
    
    return violations 


In [ ]:
def detect_put_call_parity_violations(call_prices, put_prices, S, r):
    violations = []
    
    common_strikes = call_prices.index.intersection(put_prices.index)
    common_expirations = call_prices.columns.intersection(put_prices.columns)
    
    for exp in common_expirations:
        T = exp
        for strike in common_strikes:
            call_price = call_prices.loc[strike, exp]
            put_price = put_prices.loc[strike, exp]
            
            if pd.notna(call_price) and pd.notna(put_price):
                lhs = call_price - put_price
                rhs = S - strike * np.exp(-r * T)
                violation = lhs - rhs
                
                if abs(violation) > S:
                    violations.append(f"Expiration {exp} and strike {strike} violates PCP")
    
    return violations 


In [ ]:
find_arbitrage = False 
if find_arbitrage:
    S,r = get_spot_and_risk_free_rate()
    
    call_prices = iv_to_price(IV_surface_calls, S, r, option_type='call')
    put_prices = iv_to_price(IV_surface_puts, S, r, option_type='put')

    calendar_violations_calls = detect_calendar_arbitrage(call_prices)
    calendar_violations_puts = detect_calendar_arbitrage(put_prices)
    
    vertical_violations_calls = detect_vertical_arbitrage(call_prices)
    vertical_violations_puts = detect_vertical_arbitrage(put_prices)

    butterfly_violations_calls = detect_butterfly_arbitrage(call_prices)
    butterfly_violations_puts = detect_butterfly_arbitrage(put_prices)

    pcp_violations = detect_put_call_parity_violations(call_prices, put_prices, S, r)

    total_violations = (len(calendar_violations_calls) + len(calendar_violations_puts) +
                       len(vertical_violations_calls) + len(vertical_violations_puts) +
                       len(butterfly_violations_calls) + len(butterfly_violations_puts) +
                       len(pcp_violations))

    print(f"# calendar arbitrages found: {len(calendar_violations_calls)+len(calendar_violations_puts)}")
    print(f"# vertical arbitrages found: {len(vertical_violations_calls)+len(vertical_violations_puts)}")
    print(f"# butterfly arbitrages found: {len(butterfly_violations_calls)+len(butterfly_violations_puts)}")
    print(f"put call violations found: {len(pcp_violations)}") 
    print(f"Total violations: {total_violations}")

# calendar arbitrages found: 0
# vertical arbitrages found: 0
# butterfly arbitrages found: 0
put call violations found: 0
Total violations: 0
